In [111]:
import json
import string
import tensorflow as tf
import numpy as np

In [112]:
model = tf.keras.models.load_model('saved_model/1')

In [113]:
with open("word-dict.json") as file:
    global word_dict
    word_dict = json.load(file)

In [114]:
def text_preprocessing(text, dict_word, oov_token="<OOV>", max_length=73):
    words = text.translate(str.maketrans('', '', string.punctuation)).lower().split(" ")
    result = [dict_word.get(word, dict_word[oov_token]) for word in words]
    padding = [0 for _ in range(max_length - len(result))]
    return [result + padding]

def model_predict(text_processed):
    labels = ["negative", "neutral", "positive"]
    data_pred = model.predict(text_processed)
    print("data_pred =>", data_pred)
    index_pred = np.argmax(data_pred)
    print("index_pred =>", index_pred)
    label_pred = labels[index_pred]
    print("label_pred =>", label_pred)

text_processed = text_preprocessing("tidak enak keasinan.", dict_word=word_dict)
model_predict(text_processed)

1/1 [==============================] - 1s 582ms/step
data_pred => [[0.28500882 0.21683519 0.49815592]]
index_pred => 2
label_pred => positive


In [116]:
import requests

def test_predict_tfserving(text, endpoint):
    text_processed = text_preprocessing(text, dict_word=word_dict)

    json_data = json.dumps({"instances": text_processed})

    response = requests.post(endpoint, data=json_data)
    prediction = response.json()["predictions"][0]
    print(f"Prediction from TFServing: {prediction}")

    list_class_name = ["negative", "neutral", "positive"]

    index_class_predicted = np.argmax(prediction)
    print(f"Index class predicted: {index_class_predicted}")
    print(f"Class name predicted: {list_class_name[index_class_predicted]}")

In [128]:
test_predict_tfserving(
    text="seblaknya tidak enak, tempatnya jorok banyak lalat mahall",
    endpoint="https://kulinerin-q6za24kwsq-uc.a.run.app/v1/models/kulinerin:predict"
)

Prediction from TFServing: [0.242602661, 0.203741908, 0.553655505]
Index class predicted: 2
Class name predicted: positive
